In [60]:
import pandas as pd
import numpy as np
import json
from itertools import product
import datetime
import time
from math import erf, sqrt

from ib_insync import *

print(f"started at {time.strftime('%X')}")

host = data['common']['host']
port = data[market]['port']
cid = 1

symbol = 'FDX'

with IB().connect(host=host, port=port, clientId=cid) as ib:
    c = ib.qualifyContracts(Stock(symbol, 'SMART', 'USD'))[0] # ASYNC THIS
    
    # ASYNC THIS
    ohlc = ib.reqHistoricalData(contract=c, endDateTime = '',
                       durationStr='365 D', barSizeSetting= '1 day',
                       whatToShow = 'Trades', useRTH=True)
    
    # reverse sort to have latest date on top
    df_ohlc = util.df(ohlc).sort_index(ascending=False).reset_index(drop=True)
    
    df_ohlc.insert(0, 'symbol', c.symbol)

    df_ohlc['rise'] = [df_ohlc['close'].rolling(i).apply(lambda x: x[0]-x[-1], raw=True).max()
                       for i in range(1, len(df_ohlc)+1)]
    
    df_ohlc['rise'] = df_ohlc['rise'].abs()

    df_ohlc['fall'] = [df_ohlc['close'].rolling(i).apply(lambda x: x[0]-x[-1], raw=True).min()
                       for i in range(1, len(df_ohlc)+1)]
    df_ohlc.fall = df_ohlc.fall.abs()

    df_ohlc = df_ohlc.assign(sd=df_ohlc.close.expanding(1).std(ddof=0))
    df_ohlc.sd = df_ohlc.sd.expanding(1).max() # roll the standard deviation upwards

    undPrice = {c.symbol: ib.reqTickers(c)[0].marketPrice()} # ASYNC THIS
    

    # ASYNC THIS
    chains = {c.symbol: ib.reqSecDefOptParams(
        underlyingSymbol=c.symbol, futFopExchange='',
        underlyingSecType=c.secType, underlyingConId=c.conId)[0]}
    sek = {b for a in [list(product([k], m.expirations, m.strikes))
                       for k, m in chains.items()] for b in a}

    dfc = pd.DataFrame(list(sek), columns=['symbol', 'expiry', 'strike'])
    dfc = dfc.assign(dte=[(util.parseIBDatetime(
        dt)-datetime.datetime.now().date()).days for dt in dfc.expiry])    
    dfc = dfc[dfc.dte <= data['common']['maxdte']] # Limit to max and min dte
    dfc = dfc[dfc.dte >= data['common']['mindte']]
    dfc = dfc.join(dfc.dte.apply(lambda x: df_ohlc.iloc[x][['rise', 'fall', 'sd']])) # integrate rise, fall and stdev

    # remove the calls and puts whose strike is in the threshold of st dev
    dfc['undPrice'] = undPrice[c.symbol]
    dfc = dfc.assign(right=np.where(dfc.strike >= dfc.undPrice, 'C', 'P'))
    c_mask = (dfc.right == 'C') & (dfc.strike > dfc.undPrice + data['common']['callstdmult']*dfc.sd)
    p_mask = (dfc.right == 'P') & (dfc.strike < dfc.undPrice - data['common']['putstdmult']*dfc.sd)
    dfc = dfc[c_mask | p_mask].reset_index(drop=True)

    # Based on filter selection in json weed out...
    dfc = dfc.assign(strikeRef = np.where(dfc.right == 'P', 
                                          dfc.undPrice-dfc.fall, 
                                          dfc.undPrice+dfc.rise))

    if data['common']['callRise']:
        dfc = dfc[~((dfc.right == 'C') & (dfc.strike < dfc.strikeRef))].reset_index(drop=True)

    if data['common']['putFall']:
        dfc = dfc[~((dfc.right =='P') & (dfc.strike > dfc.strikeRef))].reset_index(drop=True)

    if data['common']['onlyPuts']:
        dfc = dfc[dfc.right == 'P'].reset_index(drop=True)

    # limit to nBands
    nBand = data['common']['nBand']
    gb = dfc.groupby(['right'])

    if 'C' in [k for k in gb.indices]:
        df_calls = gb.get_group('C').reset_index(drop=True).sort_values(['symbol', 'dte', 'strike'], ascending=[True, True, True])
        df_calls = df_calls.groupby(['symbol', 'dte']).head(nBand)
    else:
        df_calls = pd.DataFrame([])

    if 'P' in [k for k in gb.indices]:
        df_puts = gb.get_group('P').reset_index(drop=True).sort_values(['symbol', 'dte', 'strike'], ascending=[True, True, False])
        df_puts = df_puts.groupby(['symbol', 'dte']).head(nBand)
    else:
        df_puts =  pd.DataFrame([])

    dfc = pd.concat([df_puts, df_calls]).reset_index(drop=True)

    # qualify the options
    opts = [Option(i.symbol, i.expiry, i.strike, i.right, data[market]['exchange']) for i in dfc[['symbol', 'expiry', 'strike', 'right']].itertuples()]
    qual_opts = ib.qualifyContracts(*opts) # ASYNC THIS

    df_qo = util.df(qual_opts).iloc[:, 1:6].rename(columns={'lastTradeDateOrContractMonth': 'expiry'})
    df_qo = df_qo.set_index(['symbol', 'expiry', 'strike', 'right']).join(
            dfc.set_index(['symbol', 'expiry', 'strike', 'right'])).reset_index()
            
    qo_ticks = ib.reqTickers(*qual_opts) # ASYNC THIS
    optPrice = {q.contract.conId: q.marketPrice() for q in qo_ticks}
    df_qo = df_qo.assign(optPrice = [optPrice[cid] for cid in df_qo.conId])

    # get the margins
    mgn_ords = [Order(action='SELL', orderType='MKT', totalQuantity=1, whatIf=True) for _ in range(len(qual_opts))]

    q_margins =[ib.whatIfOrder(qual_opts[i], mgn_ords[i]) for i in range(len(qual_opts))] # ASYNC THIS

    margins = [abs(float(m.initMarginChange)) for m in q_margins] # makes negative margins positive
    comm = [m.maxCommission for m in q_margins]
    df_qo = df_qo.assign(margin = margins, comm = comm)
    df_qo = df_qo[df_qo.margin < 1.7e7] # remove too high margin errors
    df_qo['lot'] = 100 # needs to be changed for NSE!

    df_qo = df_qo.assign(PoP=[erf(i/sqrt(2.0)) for i in abs(df_qo.strike-df_qo.undPrice)/df_qo.sd],
                         RoM = abs((df_qo.optPrice*df_qo.lot-df_qo.comm)/df_qo.margin*365/df_qo.dte))

    end = datetime.datetime.now()-start
    print(f"finished at {time.strftime('%X')} in {end.microseconds*1e-6:.2f} seconds")

started at 10:23:48
Error 200, reqId 524919: No security definition has been found for the request, contract: Option(symbol='FDX', lastTradeDateOrContractMonth='20200117', strike=138.0, right='P', exchange='SMART')
Error 200, reqId 524920: No security definition has been found for the request, contract: Option(symbol='FDX', lastTradeDateOrContractMonth='20200117', strike=137.0, right='P', exchange='SMART')
Error 200, reqId 524921: No security definition has been found for the request, contract: Option(symbol='FDX', lastTradeDateOrContractMonth='20200117', strike=136.0, right='P', exchange='SMART')
Error 200, reqId 524922: No security definition has been found for the request, contract: Option(symbol='FDX', lastTradeDateOrContractMonth='20200124', strike=131.0, right='P', exchange='SMART')
Error 200, reqId 524924: No security definition has been found for the request, contract: Option(symbol='FDX', lastTradeDateOrContractMonth='20200124', strike=129.0, right='P', exchange='SMART')
Error

In [ ]:
with IB().connect(host=host, port=port, clientId=cid) as ib:

In [61]:
df_qo

,symbol,expiry,strike,right,conId,dte,rise,fall,sd,undPrice,strikeRef,optPrice,margin,comm,lot,PoP,RoM
0,FDX,20191213,145.0,P,389803576,4,13.51,37.11,2.394806,153.29,116.18,0.44,1386.91,1.596043,100,0.999463,2.789915
1,FDX,20191213,144.0,P,389803558,4,13.51,37.11,2.394806,153.29,116.18,0.35,1305.47,1.596002,100,0.999895,2.334879
2,FDX,20191213,143.0,P,389803551,4,13.51,37.11,2.394806,153.29,116.18,0.29,1227.65,1.595974,100,0.999983,2.036914
3,FDX,20191220,141.0,P,394869462,11,23.98,67.53,3.615123,153.29,85.76,1.82,1599.84,1.598442,100,0.999325,3.741656
4,FDX,20191220,140.0,P,388155860,11,23.98,67.53,3.615123,153.29,85.76,1.69,1546.61,1.598158,100,0.999763,3.591531
5,FDX,20191220,139.0,P,394869459,11,23.98,67.53,3.615123,153.29,85.76,1.48,1495.23,1.597912,100,0.999923,3.248923
6,FDX,20191227,141.0,P,392654012,18,26.08,72.53,3.615123,153.29,80.76,2.17,1608.28,1.598901,100,0.999325,2.715855
7,FDX,20191227,140.0,P,391429474,18,26.08,72.53,3.615123,153.29,80.76,1.97,1560.79,1.598585,100,0.999763,2.538654
8,FDX,20191227,139.0,P,394200324,18,26.08,72.53,3.615123,153.29,80.76,1.78,1509.42,1.598321,100,0.999923,2.369807
9,FDX,20200103,140.0,P,392654258,25,25.17,75.11,4.089853,153.29,78.18,2.23,1548.07,1.599078,100,0.998844,2.088054


In [ ]:
df_qo.dtypes